In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local[*]").getOrCreate()

In [3]:
from pyspark.sql import functions as f
from pyspark.sql import Window

In [4]:
data = sparkSession.read.parquet("/data/sample264")

In [5]:
data.take(10)

[Row(userId=13065, trackId=944906, artistId=978428, timestamp=1501588527),
 Row(userId=101897, trackId=799685, artistId=989262, timestamp=1501555608),
 Row(userId=215049, trackId=871513, artistId=988199, timestamp=1501604269),
 Row(userId=309769, trackId=857670, artistId=987809, timestamp=1501540265),
 Row(userId=397833, trackId=903510, artistId=994595, timestamp=1501597615),
 Row(userId=501769, trackId=818149, artistId=994975, timestamp=1501577955),
 Row(userId=601353, trackId=958990, artistId=973098, timestamp=1501602467),
 Row(userId=710921, trackId=916226, artistId=972031, timestamp=1501611582),
 Row(userId=6743, trackId=801006, artistId=994339, timestamp=1501584964),
 Row(userId=152407, trackId=913509, artistId=994334, timestamp=1501571055)]

In [6]:
userTrack = data.groupBy('userId', 'trackId').count()

In [7]:
userTrack.take(10)

[Row(userId=185109, trackId=870292, count=4),
 Row(userId=93053, trackId=915614, count=1),
 Row(userId=55026, trackId=949518, count=1),
 Row(userId=640605, trackId=841340, count=3),
 Row(userId=103552, trackId=942680, count=3),
 Row(userId=227285, trackId=944606, count=2),
 Row(userId=105324, trackId=928370, count=1),
 Row(userId=647294, trackId=887536, count=1),
 Row(userId=324195, trackId=821053, count=1),
 Row(userId=513364, trackId=857897, count=10)]

In [8]:
def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(f.col(field).desc())
        
    topsDF = df.withColumn('row_number', f.row_number().over(window)) \
        .filter(f.col('row_number') <= n) \
        .drop(f.col('row_number')) 
        
    tmpDF = topsDF.groupBy(f.col(key1)).agg(f.col(key1), f.sum(f.col(field)).alias('sum_' + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, 'inner') \
        .withColumn('norm_' + field, f.col(field) / f.col('sum_' + field)) \
        .cache()

    return normalizedDF

In [9]:
userTrackNorm = norm(userTrack, 'userId', 'trackId', 'count', 1000) \
        .select('userId', 'trackId', 'norm_count')

In [10]:
userTrackNorm.take(30)

[Row(userId=3175, trackId=947718, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=940951, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=845631, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=864690, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=831005, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=930432, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=965012, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=858940, norm_count=0.1111111111111111),
 Row(userId=3175, trackId=829307, norm_count=0.1111111111111111),
 Row(userId=5518, trackId=961148, norm_count=0.5),
 Row(userId=5518, trackId=873588, norm_count=0.3333333333333333),
 Row(userId=5518, trackId=930964, norm_count=0.16666666666666666),
 Row(userId=5803, trackId=810419, norm_count=1.0),
 Row(userId=6654, trackId=802183, norm_count=0.2),
 Row(userId=6654, trackId=886091, norm_count=0.2),
 Row(userId=6654, trackId=825094, norm_count=0.2),
 Row(userId=6654, 

In [11]:
window = Window.orderBy(f.col('norm_count').desc())
    
userTrackList = userTrackNorm.withColumn('position', f.rank().over(window)) \
    .filter(f.col('position') < 40) \
    .orderBy('userId', 'trackId') \
    .select('userId', 'trackId') \
    .take(40)

In [12]:
for val in userTrackList:
    print "%s %s" % val

66 965774
116 867268
128 852564
131 880170
195 946408
215 860111
235 897176
300 857973
321 915545
328 943482
333 818202
346 864911
356 961308
428 943572
431 902497
445 831381
488 841340
542 815388
617 946395
649 901672
658 937522
662 881433
698 935934
708 952432
746 879259
747 879259
776 946408
784 806468
806 866581
811 948017
837 799685
901 871513
923 879322
934 940714
957 945183
989 878364
999 967768
1006 962774
1049 849484
1057 920458
